# Problem:
## Given a morphology in [Arbor](https://arbor.readthedocs.io/en/latest/index.html#) how does one setup the channels?

### 1. First the arbor-morphology is created
See `PyNN_neuroml_morphology_IN_Arbor.ipynb` for description of the steps.

In [1]:
from neuroml import Morphology, Segment, Point3DWithDiam as P
from pyNN.morphology import NeuroMLMorphology

In [2]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)
neuroml_morph = Morphology(segments=(soma, dend))
pynn_morph = NeuroMLMorphology(neuroml_morph)

In [3]:
import arbor
import re

In [4]:
# methods already defined in PyNN_neuroml_morphology_IN_Arbor.ipynb
def create_arbor_tree(pynn_nml_morph):
    tree = arbor.segment_tree()
    for i, nml_seg in enumerate(pynn_nml_morph.segments):
        append_arbor_tree(tree, nml_seg)
    return tree
    
def append_arbor_tree(tree, nml_seg):
    if not nml_seg.parent:
        tree.append(arbor.mnpos,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter/2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))
    else:
        tree.append(nml_seg.parent.id,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter/2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))

def get_swc_tag(nml_seg):
    if re.search("soma", nml_seg.name, re.IGNORECASE):
        return 1
    elif re.search("axon", nml_seg.name, re.IGNORECASE):
        return 2
    elif re.search("dend", nml_seg.name, re.IGNORECASE):
        return 3
    else:
        return 5

In [5]:
tree = create_arbor_tree(pynn_morph)

In [6]:
#Arbor uses a domains specific language (DSL) to describe regions and locations, which are given labels.
def create_region_definitions(pynn_morph):
    dict_defs = {}
    for i, nml_seg in enumerate(pynn_morph.segments):
        dict_defs.update({nml_seg.name: "(tag "+ str(get_swc_tag(nml_seg))+ ")"})
    dict_defs.update({"everywhere": "(all)"})
    return dict_defs

In [7]:
#dict_defs => {'soma': '(tag 1)', 'dendrite': '(tag 3)'}
tree_labels = arbor.label_dict(create_region_definitions(pynn_morph))

### 2. Create cell

In [8]:
# Cell builders need to refer to regions and locations on a cell morphology.
cell = arbor.cable_cell(tree, tree_labels)

### 3. Set cell properties.

There are [four cable properties](https://arbor.readthedocs.io/en/pydoc/cable_cell.html?#cable-properties) that are defined everywhere on all cables:

* `Vm`: Initial membrane voltage [mV].
* `cm`: Membrane capacitance [F/m²].
* `rL`: Axial resistivity of cable [Ω·cm].
* `tempK`: Temperature [Kelvin].

In [9]:
# Set cell-wide properties that will be applied by default to # the entire cell.
#cell.set_properties(Vm=-70, cm=0.02, rL=30, tempK=30+273.5)
cell.set_properties(cm=1.0, rL=500.)

### 4. Set ion properties.
##### 4.1. Set reversal potentials for respective ions.
Although, one might think of this as part of the `arbor.mechanism` it is not. But, one may set the reversal potential for respective ion in the cell created above as follows,

In [10]:
cell.set_ion(ion="na", rev_pot=50.)
cell.set_ion(ion="k", rev_pot=-77.)

### 5. Describe the desired `mechanism`.
`arbor.mechanism` describes physical processes, distributed over the membrane of the cell. There are broadly [three classes of mechanisms](https://arbor.readthedocs.io/en/pydoc/mechanisms.html?#mechanism)
- Density mechanisms are associated with regions of the cell, whose dynamics are a function of the cell state and their own state where they are present.
- Point mechanisms are defined at discrete locations on the cell, which receive events from the network.
- A third, specific type of density mechanism, which describes ionic reversal potential behaviour, can be specified for cells or the whole model.

Note: In Arbor, the reversal potential can also be [computed based on Nernst equation](https://arbor.readthedocs.io/en/pydoc/cable_cell.html?#ion-species)

#### 5.2. Set channel mechanisms.
##### 5.2.1. For leaky channel, i.e. `passive`
For `passive` channel mechanism an instance of `arbor.mechanism('passive')` is created. But, this can be done in various ways:
* Create instance using default parameter values (set in NMODL file)
    - `pas = arbor.mechanism('passive')`
* Create instance with custom reversal potential (Arbor term: [global](https://arbor.readthedocs.io/en/pydoc/mechanisms.html?#mechanism-catalogues))
    - `pas = arbor.mechanism('passive/el=-54.3')`
    - NOTE: in `arbor.mechanism` **el** stands for reversal potential
* Create instance with custom maximum conductance (Arbor term: [range](https://arbor.readthedocs.io/en/pydoc/mechanisms.html?#mechanism-catalogues))
    - `pas = arbor.mechanism('passive', {'g': 0.0003})`
    - NOTE: in `argbor.mechanism` **g** stands for gbar for respective HH-based channel, maximum conductance.
* Create instance with custom reversal potential and custom maximum conductance
    - `pas = arbor.mechanism('passive/el=-54.3', {'g': 0.0003})`
    - Alternatively, this can be done in two steps as
        - `pas = arbor.mechanism('passive/el=-54.3')`
        - `pas.set('g', 0.0003)`

For this example we would like to enforce `pas={"conductance_density": uniform('all', 0.0003), "e_rev":-54.3}`

In [11]:
leaky_chnl = arbor.mechanism('passive/el=-54.3', {'g': 0.0003})

##### 5.2.2. For HH-based Na channel, i.e. `na` and HH-based K channel, i.e. `k`

In [12]:
na_chnl = arbor.mechanism("hh", {"ena": 50., "gnabar": 0.120})
# na={"conductance_density": uniform('soma', 0.120), "e_rev": 50.0}
na_chnl = arbor.mechanism("hh", {"ek": -77., "gkbar": 0.036})
# kdr={"conductance_density": uniform('soma', 0.036), "e_rev": -77.0}
hh_na_k_chnl = arbor.mechanism("hh", {"ena": 50., "gnabar": 0.120, "ek": -77., "gkbar": 0.036, "gl": 0.})

Note that in `hh_na_k_chnl` the value for `"gl"` is set to 0 because our plan is to paint this channel to specific regions of the cell while pain `leaky_chnl` to every region. And, remember that `leaky_chnl` is also a Hodgkin-Huxley based leaky channel therefore painting it twice will result in duplicated passive channels.

### 6. Attaching the `mechanism` to the cell.
In Arbor attaching to a cell is referred to as [decoration](https://arbor.readthedocs.io/en/pydoc/cable_cell.html?#decoration). On can decorate a cell by _painting_ or _placing_.

| Painting                    | Placing                       |
|:----------------------------|:------------------------------|
| to regions of a cell to set | to locations on a cell to set |
| * Cable properties          | * Synapses                    |
| * Density mechanisms        | * Gap junction sites          |
| * Ion species               | * Threshold (spike) detectors |
|                             | * Stimuli                     |
|                             | * Probes                      |

For our purpose of attaching `arbor.mechanism` one must therefore paint.

In [13]:
cell.paint("soma", hh_na_k_chnl)

Because of the resolution options of _painting_.

| to paint               | region | cell | global |
|:-----------------------|:------:|:----:|:------:|
| cable properties       | ✓      |  ✓  |  ✓     |
| ion initial conditions | ✓      |  ✓  |  ✓     |
| density mechnism       | ✓      | -   | -       |
| ion rev pot mechanism  | -      |  ✓  | ✓      |
| ion valence            | -      | -   |  ✓     |

if the objective is to attach the `passive` mechanism to the whole cell one must loop through each segment.

In [14]:
cell.paint("everywhere", leaky_chnl)
#
#for i, seg in enumerate(pynn_morph.segments):
#    #print(seg.name)
#    cell.paint(seg.name, leaky_chnl)

```
cell_class = sim.MultiCompartmentNeuron
cell_class.label = "ExampleMultiCompartmentNeuron"
cell_class.ion_channels = {'pas': sim.PassiveLeak, 'na': sim.NaChannel, 'kdr': sim.KdrChannel}

cell_type = cell_class(morphology=NeuroMLMorphology(Morphology(segments=(soma, dend))),  # yuck
                       cm=1.0,
                       Ra=500.0,
                       ionic_species={
                              "na": IonicSpecies("na", reversal_potential=50.0),
                              "k": IonicSpecies("k", reversal_potential=-77.0)
                       },
                       pas={"conductance_density": uniform('all', 0.0003),
                            "e_rev":-54.3},
                       na={"conductance_density": uniform('soma', 0.120),
                           "e_rev": 50.0},
                       kdr={"conductance_density": uniform('soma', 0.036),
                            "e_rev": -77.0}
                       )
```

In [15]:
#cell.set_properties(Vm=-70, cm=0.02, rL=30, tempK=30+273.5)
cell.set_properties(Vm=-70)

In [16]:
from pyNN.parameters import IonicSpecies

In [17]:
ionic_species={"na": IonicSpecies("na", reversal_potential=50.0), "k": IonicSpecies("k", reversal_potential=-77.0)}

In [18]:
ionic_species["na"].ion_name

'na'

## `~/pyNN/arbor/standardmodels/ion_channels.py`

In [19]:
from pyNN.standardmodels import ion_channels as standard, build_translations

In [20]:
# arbor/standardmodels/ion_channels.py
class NaChannel(standard.NaChannel):
    """
    Exactly the same as neuron.NaChannel because
    https://arbor.readthedocs.io/en/pydoc/mechanisms.html?highlight=huxley#density-mechanisms
    https://arbor.readthedocs.io/en/pydoc/nmodl.html#nmodl
    """
    translations = build_translations(("conductance_density", "gnabar"), #(pynn_name, sim_name)
                                      ("e_rev", "ena"),)
    #variable_translations = {"m": ("hh", "m"),
    #                         "h": ("hh", "h"),}
    model = "hh"
    conductance_density_parameter = "gnabar"
    
class KdrChannel(standard.KdrChannel):
    translations = build_translations(('conductance_density', 'gkbar'),
                                      ('e_rev', 'ek'),)
    #variable_translations = {'n': ('hh', 'n')}
    model = "hh"
    conductance_density_parameter = 'gkbar'


class PassiveLeak(standard.PassiveLeak):
    translations = build_translations(('conductance_density', 'g'),
                                      ('e_rev', 'e'),)
    model = "pas"
    conductance_density_parameter = 'g'
    
class LeakyChannel(object):
    translations = build_translations(('conductance_density', 'gl'),
                                      ('e_rev', 'el'),)
    model = "hh"
    conductance_density_parameter = 'gl'

## ~/pyNN/arbor/standardmodels/cells.py

In [21]:
from pyNN.standardmodels import cells as base_cells, build_translations

In [28]:
# ~/pyNN/arbor/standardmodels/cells.py
class MultiCompartmentNeuron(base_cells.MultiCompartmentNeuron):
    """
    """
    translations = build_translations(('Ra', 'rL'), #(pynn_name, sim_name)
                                      ('cm', 'cm'),
                                      ('morphology', 'morphology'),
                                      ('ionic_species', 'ionic_species'))
    default_initial_values = {}
    ion_channels = {}
    post_synaptic_entities = {}
    
    def __init__(self, **parameters):
        # replace ion channel classes with instantiated ion channel objects
        for name, ion_channel in self.ion_channels.items():
            self.ion_channels[name] = ion_channel(**parameters.pop(name))
        # ditto for post synaptic responses
        for name, pse in self.post_synaptic_entities.items():
            self.post_synaptic_entities[name] = pse(**parameters.pop(name))
        super(arbor_MultiCompartmentNeuron, self).__init__(**parameters)
        for name, ion_channel in self.ion_channels.items():
            self.parameter_space[name] = ion_channel.parameter_space
        for name, pse in self.post_synaptic_entities.items():
            self.parameter_space[name] = pse.parameter_space
        #
        self.extra_parameters = {}
        self.spike_source = None


In [29]:
#cell_class = sim.MultiCompartmentNeuron
#cell_class.label = "ExampleMultiCompartmentNeuron"
#cell_class.ion_channels = {'pas': sim.PassiveLeak, 'na': sim.NaChannel, 'kdr': sim.KdrChannel}
cell_class = MultiCompartmentNeuron
cell_class.label = "ExampleMultiCompartmentNeuronInArbor"
cell_class.ion_channels = {'pas': PassiveLeak, 'na': NaChannel, 'kdr': KdrChannel}

In [30]:
cell_class.ion_channels

{'pas': __main__.arbor_PassiveLeak,
 'na': __main__.arbor_NaChannel,
 'kdr': __main__.arbor_KdrChannel}

In [31]:
from pyNN.morphology import uniform

In [32]:
#morphology=NeuroMLMorphology(Morphology(segments=(soma, dend)))
#neuroml_morph = Morphology(segments=(soma, dend))
#pynn_morph = NeuroMLMorphology(neuroml_morph)
cell_type = cell_class(morphology=pynn_morph,
                       cm=1.0,
                       Ra=500.0,
                       ionic_species={"na": IonicSpecies("na", reversal_potential=50.0),
                                      "k": IonicSpecies("k", reversal_potential=-77.0)},
                       pas={"conductance_density": uniform('all', 0.0003), "e_rev":-54.3},
                       na={"conductance_density": uniform('soma', 0.120), "e_rev": 50.0},
                       kdr={"conductance_density": uniform('soma', 0.036), "e_rev": -77.0}
                       )

In [33]:
cell_type.ion_channels

{'pas': arbor_PassiveLeak(<parameters>),
 'na': arbor_NaChannel(<parameters>),
 'kdr': arbor_KdrChannel(<parameters>)}

In [34]:
cell_type.parameter_space

<ParameterSpace morphology, cm, Ra, ionic_species, pas, na, kdr, shape=None>

## ~/pyNN/arbor/simulator.py

In [35]:
from pyNN import __path__ as pyNN_path
from pyNN import common
from pyNN.morphology import MorphologyFilter
import logging
import numpy
import os.path
#from neuron import h, nrn_dll_loaded
import arbor
from operator import itemgetter

In [36]:
logger = logging.getLogger("PyNN")
name = "Arbor"  # for use in annotating output data

# Instead of starting the projection var-GID range from 0, the first _MIN_PROJECTION_VARGID are 
# reserved for other potential uses
_MIN_PROJECTION_VARGID = 1000000 

## ~/pyNN/arbor/populations.py

In [37]:
import numpy
import logging
from pyNN import common
from pyNN.parameters import ArrayParameter, Sequence, ParameterSpace, simplify, LazyArray
from pyNN.standardmodels import StandardCellType
from pyNN.random import RandomDistribution, NativeRNG
#from . import simulator
#from .recording import Recorder

In [38]:
logger = logging.getLogger("PyNN")

#### Copy-paste below the generic classes in `population.py`
* `PopulationMixin`
* `Assembly`
* `PopulationView`

In [40]:
class PopulationMixin(object):

    def _set_parameters(self, parameter_space):
        """parameter_space should contain native parameters"""
        parameter_space.evaluate(mask=numpy.where(self._mask_local)[0])
        for cell, parameters in zip(self, parameter_space):
            for name, val in parameters.items():
                setattr(cell._cell, name, val)

    def _get_parameters(self, *names):
        """
        return a ParameterSpace containing native parameters
        """
        parameter_dict = {}
        for name in names:
            if name == 'spike_times':  # hack
                parameter_dict[name] = [Sequence(getattr(id._cell, name)) for id in self]
            else:
                val = numpy.array([getattr(id._cell, name) for id in self])
                if isinstance(val[0], tuple) or len(val.shape) == 2:
                    val = numpy.array([ArrayParameter(v) for v in val])
                    val = LazyArray(simplify(val), shape=(self.local_size,), dtype=ArrayParameter)
                    parameter_dict[name] = val
                else:
                    parameter_dict[name] = simplify(val)
                parameter_dict[name] = simplify(val)
        return ParameterSpace(parameter_dict, shape=(self.local_size,))

    def _set_initial_value_array(self, variable, initial_values):
        if initial_values.is_homogeneous:
            value = initial_values.evaluate(simplify=True)
            for cell in self:  # only on local node
                setattr(cell._cell, "%s_init" % variable, value)
        else:
            if isinstance(initial_values.base_value, RandomDistribution) and initial_values.base_value.rng.parallel_safe:
                local_values = initial_values.evaluate()[self._mask_local]
            else:
                local_values = initial_values[self._mask_local]            
            for cell, value in zip(self, local_values):
                setattr(cell._cell, "%s_init" % variable, value)


class Assembly(common.Assembly):
    __doc__ = common.Assembly.__doc__
    #_simulator = simulator


class PopulationView(common.PopulationView, PopulationMixin):
    __doc__ = common.PopulationView.__doc__
    #_simulator = simulator
    _assembly_class = Assembly

    def _get_view(self, selector, label=None):
        return PopulationView(self, selector, label)

In [42]:
class Population(common.Population, PopulationMixin):
    __doc__ = common.Population.__doc__
    #_simulator = simulator
    #_recorder_class = Recorder
    _assembly_class = Assembly

    def __init__(self, size, cellclass, cellparams=None, structure=None,
                 initial_values={}, label=None):
        __doc__ = common.Population.__doc__
        common.Population.__init__(self, size, cellclass, cellparams,
                                   structure, initial_values, label)
        #simulator.initializer.register(self)

    def _get_view(self, selector, label=None):
        return PopulationView(self, selector, label)
    
    def _create_cells(self):
        """
        Create cells in Arbor using the celltype of the current Population.
        """
        # this method should never be called more than once
        # perhaps should check for that
        self.first_id = simulator.state.gid_counter
        self.last_id = simulator.state.gid_counter + self.size - 1
        self.all_cells = numpy.array([id for id in range(self.first_id, self.last_id + 1)], 
                                     simulator.ID)

        # mask_local is used to extract those elements from arrays that apply to the cells on the current node
        # round-robin distribution of cells between nodes
        self._mask_local = self.all_cells % simulator.state.num_processes == simulator.state.mpi_rank

        if isinstance(self.celltype, StandardCellType):
            parameter_space = self.celltype.native_parameters
        else:
            parameter_space = self.celltype.parameter_space
        parameter_space.shape = (self.size,)
        parameter_space.evaluate(mask=None, simplify=True)
        for i, (id, is_local, params) in enumerate(zip(self.all_cells, self._mask_local, parameter_space)):
            self.all_cells[i] = simulator.ID(id)
            self.all_cells[i].parent = self
            if is_local:
                if hasattr(self.celltype, "extra_parameters"):
                    params.update(self.celltype.extra_parameters)
                self.all_cells[i]._build_cell(self.celltype.model, params)
        simulator.initializer.register(*self.all_cells[self._mask_local])
        simulator.state.gid_counter += self.size